<a href="https://colab.research.google.com/github/MarylaP/R.liniowa/blob/main/ML_Projekt_Piechowicz_poprawione.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
from sklearn.ensemble import RandomForestRegressor, AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import f1_score
from sklearn.decomposition import PCA
from sklearn import svm
from mlxtend.plotting import plot_decision_regions
import sklearn.preprocessing

from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE



In [254]:
def read_files(filelist):
  result = []
  for name in filelist:
    with open(name) as data:
        lines = data.readlines()
        l = [x[:-1] for x in lines]
        del l[::2]
    result.extend(l)
  return result

test_lines_neg = read_files(['test_neg-K.fsa','test_neg-P.fsa','test_neg-R.fsa','test_neg-T.fsa'])
test_lines_pos = read_files(['test_pos-K.fsa','test_pos-P.fsa','test_pos-R.fsa','test_pos-T.fsa'])
train_lines_pos = read_files(['train_pos-K.fsa','train_pos-P.fsa','train_pos-R.fsa','train_pos-T.fsa'])
train_lines_neg = read_files(['train_neg-K.fsa','train_neg-P.fsa','train_neg-R.fsa','train_neg-T.fsa'])

# test_lines_neg = read_files(['test_neg-P.fsa'])
# test_lines_pos = read_files(['test_pos-P.fsa'])
# train_lines_pos = read_files(['train_pos-P.fsa'])
# train_lines_neg = read_files(['train_neg-P.fsa'])


In [255]:
train_lines_pos=train_lines_pos*6

In [256]:
print(len(train_lines_neg))
print(len(train_lines_pos))
len(train_lines_neg)/len(train_lines_pos)

3974
3690


1.0769647696476965

In [257]:
data = pd.read_csv('nine_physicochemical_properties_of_amino_acid_original and corresponding molecular weight.csv', delimiter= ';', decimal = ',').set_index("Amino Acid")
data

,Hydrophobicity,Hydrophilicity,Mass,pK1,pK2,pI,Rigidity,Flexibility,Irreplaceability
Amino Acid,,,,,,,,,
A,0.62,-0.5,15,2.35,9.87,6.11,-1.34,-3.10,0.52
V,0.29,-1.0,47,1.71,10.78,5.02,-1.51,0.96,1.12
L,-0.90,3.0,59,1.88,9.60,2.98,-0.20,0.42,0.77
I,-0.74,3.0,73,2.19,9.67,3.08,-0.37,2.01,0.76
P,1.19,-2.5,91,2.58,9.24,5.91,2.88,-0.45,0.86
F,0.48,0.0,1,2.34,9.60,6.06,-1.10,-2.73,0.56
W,-0.40,-0.5,82,1.78,8.97,7.64,2.27,-0.22,0.94
M,1.38,-1.8,57,2.32,9.76,6.04,-1.74,0.42,0.65
G,-1.50,3.0,73,2.20,8.90,9.47,-1.82,3.95,0.81


In [258]:
datast = (data-data.mean())/data.std()

In [259]:
datast = datast.append(pd.DataFrame([[0]*9], columns=datast.columns, index=['X']))

In [260]:
datast

,Hydrophobicity,Hydrophilicity,Mass,pK1,pK2,pI,Rigidity,Flexibility,Irreplaceability
A,0.620140,-0.148032,-1.551628,0.776413,0.914983,-0.103339,-0.627806,-1.729144,-0.935467
V,0.290066,-0.407738,-0.516131,-2.272079,2.916509,-0.643579,-0.707393,0.401455,0.993331
L,-0.900204,1.669909,-0.127819,-1.462324,0.321124,-1.654670,-0.094101,0.118075,-0.131801
I,-0.740168,1.669909,0.325211,0.014290,0.475087,-1.605107,-0.173688,0.952473,-0.163948
P,1.190269,-1.186856,0.907678,1.871965,-0.470688,-0.202466,1.347839,-0.338482,0.157519
F,0.480109,0.111674,-2.004658,0.728780,0.321124,-0.128121,-0.515447,-1.534976,-0.806881
W,-0.400091,-0.148032,0.616444,-1.938651,-1.064548,0.654979,1.062260,-0.217783,0.414692
M,1.380312,-0.823268,-0.192538,0.633515,0.673041,-0.138034,-0.815070,0.118075,-0.517561
G,-1.500340,1.669909,0.325211,0.061923,-1.218511,1.561986,-0.852523,1.970542,-0.003215
S,1.060240,-0.823268,-0.192538,0.824046,0.321124,-0.138034,-0.815070,0.118075,-0.742587


In [261]:

def seq_to_feature_vec(input):
  sample = []
  for a in input:
    sample.append(datast.loc[a].to_numpy())
  sample = np.concatenate(sample)
  return sample

print(seq_to_feature_vec(test_lines_neg[0]))

  

[ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.         

In [262]:
def sequences_list_to_dataset(input):
  result = []
  for i in input:
    vec = seq_to_feature_vec(i)
    result.append(vec)
  return np.array(result)

In [263]:
train_pos=sequences_list_to_dataset(train_lines_pos)
train_neg=sequences_list_to_dataset(train_lines_neg)

In [264]:
test_pos=sequences_list_to_dataset(test_lines_pos)
test_neg=sequences_list_to_dataset(test_lines_neg)

In [265]:
X_train = np.concatenate([train_pos,train_neg])
Y_train = np.concatenate([np.ones(train_pos.shape[0]), np.zeros(train_neg.shape[0])])

In [266]:
len(train_neg)

3974

In [267]:
test_pos_multiplied = np.tile(test_pos, (5,1))
X_test = np.concatenate([test_pos_multiplied,test_neg])
Y_test = np.concatenate([np.ones(test_pos_multiplied.shape[0]), np.zeros(test_neg.shape[0])])

In [268]:
pca = PCA(n_components=100)
pca.fit(X_train)
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

In [269]:
#AdaBoost

In [284]:
classifier = AdaBoostClassifier(n_estimators = 500)
classifier=classifier.fit(X_train_pca, Y_train)

In [286]:
y_pred = classifier.predict(X_test_pca)
y_pred_score = classifier.predict_proba(X_test_pca)


In [287]:
y_pred


array([1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [288]:
y_pred_score

array([[0.49960009, 0.50039991],
       [0.50126706, 0.49873294],
       [0.50277551, 0.49722449],
       ...,
       [0.49954775, 0.50045225],
       [0.50187624, 0.49812376],
       [0.50436912, 0.49563088]])

In [274]:
#printuje wyniki

In [289]:
from sklearn.metrics import roc_auc_score
roc_auc_score(Y_test, y_pred_score[:,1])

0.48629292757565445

In [290]:
result = confusion_matrix(Y_test, y_pred)
print("Confusion Matrix:")
print(result)
result1 = classification_report(Y_test, y_pred)
print("Classification Report:",)
print (result1)
result2 = accuracy_score(Y_test,y_pred)
print("Accuracy:",result2)

Confusion Matrix:
[[310  36]
 [310  30]]
Classification Report:
              precision    recall  f1-score   support

         0.0       0.50      0.90      0.64       346
         1.0       0.45      0.09      0.15       340

    accuracy                           0.50       686
   macro avg       0.48      0.49      0.39       686
weighted avg       0.48      0.50      0.40       686

Accuracy: 0.4956268221574344


In [278]:
def sum(input):
    sum = 0
    for i in input:
        sum = sum + i
    return(sum)

In [279]:
sum(pca.explained_variance_ratio_)

0.8677290148716644

In [294]:
clf = svm.SVC(kernel='rbf')

clf.fit(X_train, Y_train)

y_pred_svm = clf.predict(X_test)


In [298]:
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization
from keras.metrics import AUC
#tworzę model
model = Sequential()
model.add(Dense(32, activation='relu', kernel_regularizer='l1'))
model.add(BatchNormalization())
model.add(Dense(1, activation='sigmoid', kernel_regularizer='l1'))
#wywołuję model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', AUC()])
#dopasowuje model
model.fit(X_train_pca, Y_train, epochs=300, batch_size=32)
#sprawdzam model na zbiorze testowym 
scores = model.evaluate(X_test_pca, Y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


Epoch 1/300
240/240 [==============================] - 2s 2ms/step - loss: 2.9649 - accuracy: 0.5645 - auc_5: 0.5888
Epoch 2/300
240/240 [==============================] - 0s 2ms/step - loss: 1.4307 - accuracy: 0.6670 - auc_5: 0.7236
Epoch 3/300
240/240 [==============================] - 1s 2ms/step - loss: 0.9683 - accuracy: 0.7049 - auc_5: 0.7757
Epoch 4/300
240/240 [==============================] - 1s 2ms/step - loss: 0.8414 - accuracy: 0.7266 - auc_5: 0.8004
Epoch 5/300
240/240 [==============================] - 1s 2ms/step - loss: 0.7892 - accuracy: 0.7482 - auc_5: 0.8237
Epoch 6/300
240/240 [==============================] - 0s 2ms/step - loss: 0.7605 - accuracy: 0.7700 - auc_5: 0.8467
Epoch 7/300
240/240 [==============================] - 1s 2ms/step - loss: 0.7413 - accuracy: 0.7903 - auc_5: 0.8645
Epoch 8/300
240/240 [==============================] - 0s 2ms/step - loss: 0.7275 - accuracy: 0.8025 - auc_5: 0.8809
Epoch 9/300
240/240 [==============================] - 1s 2ms/st